# Carga de ficheros de renta por sección.

En esta sección vamos a proceder a la carga de los ficheros de renta por persona y por secciones de población.

Para ello instalaremos las siguientes librerías:

In [7]:
import pandas as pd
import re
import plotly.graph_objects as go


Los datos los hemos obtenido de la página del Instituto Nacional de Estadística [INE](https://www.ine.es/welcome.shtml). Hemos decidido descargar los datos con la definición máxima posible, por secciones. El formato de la descarga ha sido en CSV, separado por ";", con decimales "," y los millares ".".

A continuación mostramos la función diseñada para cargar los CSV y generar el dataframe de trabajo:


In [4]:
bbdd = ["Alava","Albacete","Alicante","Almeria","Asturias","Avila","Badajoz","Barcelona",
        "Bizkaia","Burgos","Caceres","Cadiz","Cantabria","Islas Baleares","Castellon","Ceuta","Ciudad Real",
        "Cordoba","Coruña","Cuenca","Gipuzkoa","Girona","Granada","Guadalajara","Huelva","Huesca","Jaen","La Rioja",
        "Las Palmas","Leon","LLeida","Lugo","Madrid","Malaga","Melilla","Murcia","Navarra","Ourense","Palencia","Pontevedra",
        "Salamanca","Santa Cruz de Tenerife","Segovia","Sevilla","Soria","Tarragona","Teruel","Toledo","Valencia",
        "Valladolid","Zamora","Zaragoza"]


def get_BBDD_rentas (bbdd):
    #Carga de ficheros CSV
    base = []
    NIE = []
    renta = []
    poblacion = []
    provincias = []
    for i in bbdd:
        provincia = i
        datos = pd.read_csv('datos_renta/'+i+'.csv', index_col=False,encoding='iso-8859-1'
                            ,sep=';',skiprows=5, decimal= ',',thousands='.')
        datos.rename(columns={'Unnamed: 0':'identificador','2016':'renta'},inplace=True)
        datos.drop(datos.tail(3).index,inplace=True)
        for i in range(len(datos.identificador)):
            provincias.append(provincia)
        for i in range(len(datos.identificador)):
            NIE_value = int(re.findall("[0-9]+", datos.identificador[i])[0])
            NIE.append(NIE_value) 
        for i in range(len(datos.identificador)):
             nombre = re.findall("[a-zA-Záéíóúñ\\- /]+", datos.identificador[i])[0]
             nombre = re.sub(" sección","",nombre)
             nombre = re.sub(" secci","",nombre)
             nombre = re.sub("^ ","",nombre)
             nombre = re.sub(" $","",nombre)
             poblacion.append(nombre)
        for i in range(len(datos.identificador)):
            if datos.renta[i] == '.':
                datos.renta[i] = float('NaN')
            if isinstance(datos.renta[i],str) == True:
                renta_value = float(re.sub("\\.0","",datos.renta[i]))
                renta.append(renta_value)
            else:
                renta_value = float(datos.renta[i])
                renta.append(renta_value)
    d = {'provincia':provincias,'poblacion':poblacion,'NIE':NIE,'renta':renta}
    datos = pd.DataFrame(d)
    return datos

basedatos = get_BBDD_rentas(bbdd)
basedatos

,provincia,poblacion,NIE,renta
0,Alava,Agurain/Salvatierra,1051,12374.0
1,Alava,Alegría-Dulantzi,1001,13086.0
2,Alava,Amurrio,1002,13691.0
3,Alava,Añana,1049,15548.0
4,Alava,Aramaio,1003,17194.0
...,...,...,...,...
36284,Zaragoza,Zaragoza,5029712010,14085.0
36285,Zaragoza,Zuera,5029801001,12307.0
36286,Zaragoza,Zuera,5029801002,10848.0
36287,Zaragoza,Zuera,5029801003,10726.0


## Funcionamiento de get_BBDD_rentas
La función descrita más arriba es el resultado de usar la táctica "Divide y vencerás". Se hizo una primera limpieza para sólo un CSV y después se contruyó una función que lo hiciese para todos los CSV. Aunque no fue tan sencillo ya que el formato no es uniforme en todos los CSV del INE.

A la función sólamente debemos de pasarle una lista de las provincias de las que se quiere realizar la carga de datos. En nuestro caso, son todas los existentes, un CSV por provincia. Generamos listas vacías para acumular los datos con el formato deseado y entramos en el loop principal que recorrerá la lista insertada e irá cargando todos los ficheros.

Una vez están los ficheros cargados se almacena en ```datos```. Los datos obtenidos son muy crudos; dos columnas, una con la renta y otro con una descripción donde se incluye la población, el codigo INE y la sección. Para subsanar esta "aglomeración" de datos, se ha preparado un loop por cada valor a incluir en el dataframe final; ```provincia ```,```poblacion ```,```INE ``` y ```renta ```.

### Limpieza y formateo de los datos cargados.

#### Provincia.
Aprovechando que el loop principal ya está usando el nombre de la provincia, se crea un loop para generar una lista de longitud el dataframe final y con el nombre de la provincia repetido tantas veces como filas de datos tenga la provincia cargada.
#### INE.
Usando ```regex``` obtenemos todos los números de la columna "identificado"´de los CSV cargados, que no es más que la descripción que queremos limpiar.
#### Poblacion.
Volviendo a usar ```regex``` pero esta vez con algo más de contundencia. Primero obtenemos todos los caracteres que hemos observado que aparecen como parte del nombre de la población para seguidamente quitarle tanto "sección" como "secci" (en algunos CSV sección aparecía como secci). Finalmente retiramos los posibles espacios que existan tanto al inicio como al fin del nombre.
#### Renta.
Aparentemente la más sencilla, pero analizando bien los datos hemos comprobado que no todos siguen los mismos criterios. Existen valores vacíos, valores con un "." como valor y otros que efectivamente son dato. Hemos usado otra condición más porque en el CSV de Teruel incluyeron renta (que son valores enteros) con un ".0" al final de cada valor.

### Generación del data frame
Una vez listados todos los datos formateados en sus correspondientes listas vacías, generamos un diccionario con dichas listas y usando la funcion pd.DataFrame generamos el DF "datos".

## Impresión de tabla.
Para mostrar los datos del dataframe ```basedatos``` hemos decidido usar la librería ```plotly```. Esta librería tiene muchas opciones de personalización de una forma muy sencilla. Como añadido, posibilitamos que se pueda hacer un prefiltro (```basedatos``` tiene más de 36.000 filas).

### Prefiltro.
De una forma sencilla, posibilitamos que se pueda filtrar por provincia, población o código INE. Se selecciona una de las tres opciones y se indica el valor por el cual filtrar.
(p.e.: pruebe a introducir filtro por población ```2``` y filtre por ```Oleiros```.

Otra opción contemplada es que introduzca cualquier otro valor diferente a los descritos para que se muestre toda la tabla, ordenada por renta de mayor a menor.

### Generación de la tabla
Basicamente la función ```Table``` de la librería ```plotly.graph_objects``` permite personalizar el formato de la tabla de un DataFrame de pandas. En nuestro caso hemos modificado el encabezado ( a través de ```header```) y las celdas (a través de ```cell```). 

Se ha decidido cambiar el color tanto del encabezado como de las celdas. En las celdas en concreto se han puesto colores para las columnas pares y otros para los impares para facilitar la lectura de la tabla. De la misma forma, en el encabezado se ha modificado el tamaño y aspecto de la fuente.


In [6]:

basedatos_tabla = basedatos.sort_values('renta', ascending=False)
print("Para facilitar la visualización de la tabla vamos a realizar unos filtros:")
print("Filtro por provincia, insertar 1")
print("Filtro por poblacion, insertar 2")
print("Filtro por código INE, insertar 3")
print("Si quiere visualizar toda la tabla, introduzca cualquier otro valor")
seleccion = input()
if seleccion == "1":
    print("A continuación se muestran las provincias disponibles:")
    print(basedatos_tabla.provincia.unique())
    print("Introduzca el nombre de la provincia:")
    provincia_selected = input()
    basedatos_tabla = basedatos_tabla[basedatos_tabla['provincia'] == provincia_selected]
elif seleccion == "2":
    print("A continuación se muestran las poblaciones disponibles:")
    print(basedatos_tabla.poblacion.unique())
    print("Introduzca el nombre de la poblacion:")
    poblacion_selected = input()
    basedatos_tabla = basedatos_tabla[basedatos_tabla['poblacion'] == poblacion_selected]
elif seleccion == "3":
    print("Introduzca el código INE:")
    INE_selected = input()
    basedatos_tabla = basedatos_tabla[basedatos_tabla['NIE'] == INE_selected]
else:
    print("Se imprimirá toda la tabla")




fig = go.Figure(data=[go.Table(
                header = dict(values = ['Provincia','Población','INE','<b>Renta(€)</b>'],
                              fill = dict(color = 'goldenrod'),
                             font = dict(size = 20, color = 'white')),
                cells = dict(values = [basedatos_tabla.provincia, basedatos_tabla.poblacion,
                                      basedatos_tabla.NIE,
                                      basedatos_tabla.renta],
                            fill = dict(color = ['lemonchiffon','khaki','lemonchiffon','khaki'])))
                     ])
fig.show()

Para facilitar la visualización de la tabla vamos a realizar unos filtros:
Filtro por provincia, insertar 1
Filtro por poblacion, insertar 2
Filtro por código INE, insertar 3
Si quiere visualizar toda la tabla, introduzca cualquier otro valor
2
A continuación se muestran las poblaciones disponibles:
['Donostia/San Sebastián' 'Val' 'Getxo' ... 'Villarroya del Campo' 'Vilue'
 'Vistabella']
Introduzca el nombre de la poblacion:
Oleiros
